Modelo: Dell Inc. Dell G15 5511
Memória: 32GB
processador: 11th Gen Intel® Core™ i7-11800H @ 2.30GHz × 16
Gráfica: NVIDIA Corporation GA106M [GeForce RTX 3060 Mobile / Max-Q] / Mesa Intel® UHD Graphics (TGL GT1)
Capacidade de disco: 1TB
S.O.: Ubuntu 22.04.4 LTS
Tipo do S.O.: 64 bits

## Exercício 1 - Função 105 - Rosenbrock

Avaliar para diferentes valores de D

$$
f_{105}(\mathbf{x}) = \sum_{i=1}^{D-1} \left[100(x_{i+1} - x_i^2)^2 + (x_i - 1)^2\right]
$$

In [ ]:
import numpy as np
import time
from scipy.optimize import minimize

def rosenbrock(x):
    """Função Rosenbrock com gradiente e Hessiana"""
    D = len(x)
    fval = sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

    # Gradiente
    grad = np.zeros(D)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    if D > 2:
        grad[1:-1] = 200 * (x[1:-1] - x[:-2]**2) - 400 * (x[2:] - x[1:-1]**2) * x[1:-1] - 2 * (1 - x[1:-1])
    grad[-1] = 200 * (x[-1] - x[-2]**2)

    # Hessiana
    H = np.zeros((D, D))
    for i in range(D):
        if i > 0:
            H[i, i - 1] = -400 * x[i - 1]
        if i < D - 1:
            H[i, i] = 1200 * x[i]**2 - 400 * x[i + 1] + 2
            H[i, i + 1] = -400 * x[i]
        else:
            H[i, i] = 200
    for i in range(D - 1):
        H[i + 1, i] = H[i, i + 1]

    return fval, grad, H

# Configurações
dimensions = [2, 5, 10, 20]  # Diferentes valores de D para avaliar
n_runs = 10                   # Número de execuções para média
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: rosenbrock(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
}

all_results = []

for D in dimensions:
    # Ponto inicial aleatório para cada dimensão
    x0 = np.random.uniform(-2, 2, D)
    
    for name, opts in methods.items():
        times = []
        last_res = None
        
        for _ in range(n_runs):
            try:
                start_time = time.time()
                res = minimize(lambda x: rosenbrock(x)[0], x0, **opts, 
                              options={'maxiter': 10000, 'disp': False})
                elapsed_time = (time.time() - start_time) * 1000  # ms
                times.append(elapsed_time)
                last_res = res
            except Exception as e:
                print(f"Erro no método {name} (D={D}): {str(e)}")
                times.append(np.nan)
        
        if last_res is not None:
            avg_time = np.nanmean(times)
            all_results.append({
                'Method': name,
                'D': D,
                'f(x*)': last_res.fun,
                'nfev': last_res.nfev,
                'njev': last_res.get('njev', None),
                'nhev': last_res.get('nhev', None),
                'Time (ms)': avg_time,
                'Success': last_res.success
            })

# Apresenta os resultados com alinhamento (mesmo formato original)
print("\n==== Rosenbrock Results (Média de 10 execuções) ====\n")
header = f"{'Method':<20} | {'D':>3} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for res in all_results:
    print(f"{res['Method']:<20} | {res['D']:3} | {res['f(x*)']:12.4e} | {res['nfev']:6d} | "
          f"{res['njev'] if res['njev'] is not None else '  N/A':>6} | "
          f"{res['nhev'] if res['nhev'] is not None else '  N/A':>6} | "
          f"{res['Time (ms)']:10.2f} | {str(res['Success']):>8}")

In [ ]:
==== Rosenbrock Results (Média de tempo de 10 execuções) ====

Method               |   D |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------------
Nelder-Mead          |   2 |   2.6005e-10 |    160 |    N/A |    N/A |       4.91 |     True
BFGS                 |   2 |   7.0469e-15 |     25 |     25 |    N/A |       3.01 |     True
Newton-CG            |   2 |   3.2971e-13 |     37 |     37 |     26 |       4.74 |     True
trust-ncg            |   2 |   4.2430e-12 |     19 |     17 |     16 |       2.43 |     True
Nelder-Mead          |   5 |   5.2883e-09 |    788 |    N/A |    N/A |      34.85 |     True
BFGS                 |   5 |   3.9308e+00 |     36 |     36 |    N/A |       5.08 |     True
Newton-CG            |   5 |   3.0443e-04 |   3778 |   3778 |   3756 |     664.41 |     True
trust-ncg            |   5 |   4.9474e-09 |   3429 |   2884 |   2883 |     591.22 |     True
Nelder-Mead          |  10 |   8.9953e+00 |   1615 |    N/A |    N/A |      95.12 |     True
BFGS                 |  10 |   3.5951e-15 |    102 |    102 |    N/A |      18.05 |     True
Newton-CG            |  10 |   6.2463e-04 |   4096 |   4096 |   4065 |     891.57 |     True
trust-ncg            |  10 |   4.1628e-09 |   3266 |   3199 |   3198 |     713.97 |     True
Nelder-Mead          |  20 |   1.1509e+02 |  12508 |    N/A |    N/A |    1006.03 |    False
BFGS                 |  20 |   5.7951e-14 |    164 |    164 |    N/A |      34.24 |     True
Newton-CG            |  20 |   1.2976e-03 |   4749 |   4749 |   4717 |    1252.65 |     True
trust-ncg            |  20 |   1.4509e-09 |   3849 |   3844 |   3843 |    1087.46 |     True

## Exercício 2- Função 114 - Scahffer 3

Testar para diferentes pontos iniciais.

$$
f_{114}(\mathbf{x}) = 0.5 + \frac{\sin^2\left(\cos\left|x_1^2 - x_2^2\right|\right) - 0.5}{1 + 0.001(x_1^2 + x_2^2)^2}
$$

In [ ]:
import numpy as np
import time
from scipy.optimize import minimize

def scalifier3(x):
    """Função Scalifier 3 (apenas cálculo da função)"""
    x1, x2 = x[0], x[1]
    numerator = np.sin(np.cos(abs(x1**2 - x2**2)))**2 - 0.5
    denominator = 1 + 0.001*(x1**2 + x2**2)**2
    return 0.5 + numerator / denominator

# Configurações
n_runs = 10
bounds = [(-100, 100), (-100, 100)]
initial_points = [
    np.array([0.0, 0.0]),      # Centro
    np.array([0.0, 1.253115]), # Ponto ótimo conhecido
    np.array([50.0, 50.0]),    # Canto superior direito
    np.array([-50.0, -50.0]),  # Canto inferior esquerdo
    np.array([-80.0, 30.0]),   # Ponto aleatório 1
    np.array([20.0, -90.0])    # Ponto aleatório 2
]

methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS"},
    "L-BFGS-B": {"method": "L-BFGS-B", "bounds": bounds},
    "trust-constr": {"method": "trust-constr", "bounds": bounds},
}

# Teste para diferentes pontos iniciais
results = []

for i, x0 in enumerate(initial_points):
    print(f"\n=== Teste com ponto inicial {i+1}: {x0} ===")
    
    for name, opts in methods.items():
        times = []
        solutions = []
        func_values = []
        successes = []
        
        for _ in range(n_runs):
            try:
                # Adiciona pequena perturbação aleatória ao ponto inicial
                perturbed_x0 = x0 + np.random.normal(0, 0.1, size=2)
                
                start_time = time.time()
                res = minimize(scalifier3, perturbed_x0, **opts,
                             options={'maxiter': 10000, 'disp': False})
                elapsed_time = (time.time() - start_time) * 1000
                
                times.append(elapsed_time)
                solutions.append(res.x)
                func_values.append(res.fun)
                successes.append(res.success)
                
            except Exception as e:
                print(f"Erro no método {name}: {str(e)}")
                times.append(np.nan)
                solutions.append(np.array([np.nan, np.nan]))
                func_values.append(np.nan)
                successes.append(False)
        
        # Processa resultados
        valid_runs = [i for i in range(n_runs) if successes[i]]
        n_valid = len(valid_runs)
        
        if n_valid > 0:
            avg_time = np.mean([times[i] for i in valid_runs])
            avg_solution = np.mean([solutions[i] for i in valid_runs], axis=0)
            avg_func = np.mean([func_values[i] for i in valid_runs])
            best_func = np.min([func_values[i] for i in valid_runs])
            best_solution = solutions[np.argmin([func_values[i] for i in valid_runs])]
            
            results.append({
                'Initial Point': f"{x0[0]:.1f}, {x0[1]:.1f}",
                'Method': name,
                'Avg Time (ms)': avg_time,
                'Success Rate': f"{n_valid/n_runs*100:.1f}%",
                'Avg f(x*)': avg_func,
                'Best f(x*)': best_func,
                'Best Solution': f"[{best_solution[0]:.6f}, {best_solution[1]:.6f}]",
                'Distance to Optimum': np.linalg.norm(best_solution - np.array([0, 1.253115]))
            })

# Exibe resultados detalhados
print("\n=== RESULTADOS DETALHADOS ===")
print("="*120)
print("{:<15} {:<15} {:<15} {:<15} {:<15} {:<15} {:<25} {:<15}".format(
    'Ponto Inicial', 'Método', 'Tempo Médio', 'Taxa Sucesso', 'Média f(x*)', 
    'Melhor f(x*)', 'Melhor Solução', 'Distância Ótimo'))
print("-"*120)

for res in results:
    print("{:<15} {:<15} {:<15.2f} {:<15} {:<15.6f} {:<15.6f} {:<25} {:<15.6f}".format(
        res['Initial Point'],
        res['Method'],
        res['Avg Time (ms)'],
        res['Success Rate'],
        res['Avg f(x*)'],
        res['Best f(x*)'],
        res['Best Solution'],
        res['Distance to Optimum']))

# Análise agregada por método
print("\n=== ANÁLISE POR MÉTODO ===")
print("="*90)
print("{:<15} {:<15} {:<15} {:<15} {:<15}".format(
    'Método', 'Média Tempo', 'Taxa Sucesso', 'Melhor f(x*)', 'Distância Média'))
print("-"*90)

for method in methods.keys():
    method_results = [r for r in results if r['Method'] == method]
    if method_results:
        avg_time = np.mean([r['Avg Time (ms)'] for r in method_results])
        avg_success = np.mean([float(r['Success Rate'][:-1]) for r in method_results])
        best_f = np.min([r['Best f(x*)'] for r in method_results])
        avg_dist = np.mean([r['Distance to Optimum'] for r in method_results])
        
        print("{:<15} {:<15.2f} {:<15.1f}% {:<15.6f} {:<15.6f}".format(
            method, avg_time, avg_success, best_f, avg_dist))

In [ ]:
=== RESULTADOS DETALHADOS (Média de tempo de 10 execuções)===
========================================================================================================================
Ponto Inicial   Método          Tempo Médio     Taxa Sucesso    Média f(x*)     Melhor f(x*)    Melhor Solução            Distância Ótimo
------------------------------------------------------------------------------------------------------------------------
0.0, 0.0        Nelder-Mead     7.01            100.0%          0.001230        0.001230        [1.253002, -0.000023]     1.772108       
0.0, 0.0        BFGS            11.95           100.0%          0.103746        0.001230        [0.000009, -1.253002]     2.506117       
0.0, 0.0        L-BFGS-B        19.74           100.0%          0.001230        0.001230        [0.000004, 1.253002]      0.000113       
0.0, 0.0        trust-constr    167.52          100.0%          0.218028        0.001230        [-1.253002, -0.000000]    1.772092       
0.0, 1.3        Nelder-Mead     2.96            100.0%          0.001230        0.001230        [0.000023, 1.253002]      0.000115       
0.0, 1.3        BFGS            4.70            100.0%          0.002193        0.001230        [0.000005, 1.253002]      0.000113       
0.0, 1.3        L-BFGS-B        8.50            100.0%          0.001230        0.001230        [0.000002, 1.253002]      0.000113       
0.0, 1.3        trust-constr    119.00          100.0%          0.072091        0.001230        [-0.000000, 1.253002]     0.000113       
50.0, 50.0      Nelder-Mead     19.13           100.0%          0.449050        0.303291        [0.000057, 6.265955]      5.012840       
50.0, 50.0      BFGS            1.43            100.0%          0.499980        0.499980        [49.863636, 49.973771]    69.714307      
50.0, 50.0      L-BFGS-B        1.47            100.0%          0.499980        0.499980        [49.998651, 49.888568]    69.751504      
50.0, 50.0      trust-constr    7095.63         100.0%          0.014654        0.001230        [-0.000000, 1.253002]     0.000113       
-50.0, -50.0    Nelder-Mead     27.52           100.0%          0.448158        0.303291        [-0.000011, -6.265953]    7.519068       
-50.0, -50.0    BFGS            2.07            100.0%          0.499980        0.499980        [-49.961594, -49.819912]  71.446588      
-50.0, -50.0    L-BFGS-B        1.78            100.0%          0.499980        0.499980        [-49.991830, -49.850210]  71.489390      
-50.0, -50.0    trust-constr    6674.14         100.0%          0.028289        0.001230        [-1.253002, 0.000000]     1.772092       
-80.0, 30.0     Nelder-Mead     21.07           100.0%          0.499990        0.499983        [-73.966773, 0.000020]    73.977387      
-80.0, 30.0     BFGS            1.30            100.0%          0.499991        0.499991        [-79.836557, 29.889261]   84.816889      
-80.0, 30.0     L-BFGS-B        1.22            100.0%          0.499991        0.499991        [-79.837571, 30.049191]   84.871972      
-80.0, 30.0     trust-constr    4940.85         100.0%          0.499983        0.499979        [-69.761223, 2.855759]    69.779630      
20.0, -90.0     Nelder-Mead     3.33            100.0%          0.499993        0.499992        [21.178838, -87.719046]   91.458125      
20.0, -90.0     BFGS            1.67            100.0%          0.499993        0.499993        [19.877475, -89.877028]   93.272810      
20.0, -90.0     L-BFGS-B        1.95            100.0%          0.499993        0.499993        [19.944541, -89.891882]   93.301636      
20.0, -90.0     trust-constr    3249.31         100.0%          0.499992        0.499991        [1.986306, -86.210447]    87.486113      

=== ANÁLISE POR MÉTODO ===
==========================================================================================
Método          Média Tempo     Taxa Sucesso    Melhor f(x*)    Distância Média
------------------------------------------------------------------------------------------
Nelder-Mead     13.50           100.0          % 0.001230        29.956607      
BFGS            3.85            100.0          % 0.001230        53.626137      
L-BFGS-B        5.77            100.0          % 0.001230        53.235788      
trust-constr    3707.74         100.0          % 0.001230        26.801692  

## Exercício 3 - Função 142 - Streched V Sine Wave Function

Avaliar para diferentes valores de D

$$
f_{142}(\mathbf{x}) = \sum_{i=1}^{D-1} (x_{i+1}^2 + x_i^2)^{0.25} \left[ \sin^2 \{50(x_{i+1}^2 + x_i^2)^{0.1}\} + 0.1 \right]
$$

In [ ]:
import numpy as np
import time
from scipy.optimize import minimize

def stretched_v_sine(x):
    """Função Stretched V Sine Wave com gradiente numérico"""
    D = len(x)
    fval = 0.0
    for i in range(D-1):
        sum_sq = x[i+1]**2 + x[i]**2
        term = (sum_sq)**0.25 * (np.sin(50*(sum_sq)**0.1)**2 + 0.1)
        fval += term
    
    return fval

# Configurações
dimensions = [2, 5, 10]  # Dimensões a avaliar (reduzido devido à complexidade)
n_runs = 10               # Número de execuções para média
bounds = [(-10, 10)]      # Restrições para cada dimensão

methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS"},
    "L-BFGS-B": {"method": "L-BFGS-B", "bounds": bounds},
    "trust-constr": {"method": "trust-constr", "bounds": bounds},
}

all_results = []

for D in dimensions:
    current_bounds = bounds * D  # Aplica restrições a todas as dimensões
    
    for name, opts in methods.items():
        times = []
        func_vals = []
        distances = []
        successes = 0
        
        # Configura bounds para métodos que suportam
        if opts["method"] in ["L-BFGS-B", "trust-constr"]:
            opts["bounds"] = current_bounds
        
        for _ in range(n_runs):
            try:
                # Gera ponto inicial aleatório dentro dos bounds
                x0 = np.random.uniform(low=-10, high=10, size=D)
                
                start_time = time.time()
                res = minimize(stretched_v_sine, x0, **opts,
                             options={'maxiter': 5000, 'disp': False})
                elapsed_time = (time.time() - start_time) * 1000  # ms
                
                # Verifica se a solução está dentro dos bounds
                valid = all(-10 <= xi <= 10 for xi in res.x)
                
                times.append(elapsed_time)
                func_vals.append(res.fun if valid else np.nan)
                distances.append(np.linalg.norm(res.x) if valid else np.nan)  # Distância até o ótimo (0,0,...)
                successes += int(res.success and valid)
                
            except Exception as e:
                print(f"Erro no método {name} (D={D}): {str(e)}")
                times.append(np.nan)
                func_vals.append(np.nan)
                distances.append(np.nan)
        
        # Calcula estatísticas (ignorando NaNs)
        avg_time = np.nanmean(times)
        std_time = np.nanstd(times)
        avg_func = np.nanmean(func_vals)
        std_func = np.nanstd(func_vals)
        avg_dist = np.nanmean(distances)
        success_rate = successes/n_runs*100
        
        all_results.append({
            'Method': name,
            'D': D,
            'f(x*)': f"{avg_func:.4e} ± {std_func:.2e}",
            'Time (ms)': f"{avg_time:.1f} ± {std_time:.1f}",
            'Success Rate (%)': success_rate,
            'Distance to Optimum': f"{avg_dist:.4e}",
            'Bounds': "[-10,10]"
        })

# Exibe resultados
print("\n==== Stretched V Sine Wave Results ====")
print("{:<15} {:<5} {:<20} {:<20} {:<20} {:<20} {:<10}".format(
    'Method', 'D', 'f(x*)', 'Time (ms)', 'Success Rate (%)', 
    'Distance to Optimum', 'Bounds'))
print("-"*120)

for res in all_results:
    print("{:<15} {:<5} {:<20} {:<20} {:<20} {:<20} {:<10}".format(
        res['Method'],
        res['D'],
        res['f(x*)'],
        res['Time (ms)'],
        f"{res['Success Rate (%)']:.1f}",
        res['Distance to Optimum'],
        res['Bounds']))

# Visualização para D=2
if 2 in dimensions:
    print("\n=== Visualização para D=2 ===")
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.pyplot as plt
    
    x = np.linspace(-10, 10, 100)
    y = np.linspace(-10, 10, 100)
    X, Y = np.meshgrid(x, y)
    Z = np.zeros_like(X)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i,j] = stretched_v_sine([X[i,j], Y[i,j]])
    
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_zlabel('f(x)')
    ax.set_title('Stretched V Sine Wave Function (D=2)')
    fig.colorbar(surf)
    plt.show()

In [ ]:
==== Stretched V Sine Wave Results (Média de tempo de 10 execuções) ====
Method          D     f(x*)                Time (ms)            Success Rate (%)     Distance to Optimum  Bounds    
------------------------------------------------------------------------------------------------------------------------
Nelder-Mead     2     2.7811e-01 ± 4.37e-02 3.0 ± 0.6            90.0                 7.9254e+00           [-10,10]  
BFGS            2     2.8511e-01 ± 5.08e-02 1.9 ± 0.4            100.0                8.3862e+00           [-10,10]  
L-BFGS-B        2     2.8622e-01 ± 6.07e-02 2.2 ± 0.4            100.0                8.5600e+00           [-10,10]  
trust-constr    2     2.5291e-01 ± 5.62e-02 197.7 ± 84.0         100.0                6.7110e+00           [-10,10]  
Nelder-Mead     5     1.0672e+00 ± 1.09e-01 9.3 ± 4.8            100.0                1.1282e+01           [-10,10]  
BFGS            5     1.0231e+00 ± 1.34e-01 6.6 ± 1.3            90.0                 1.1794e+01           [-10,10]  
L-BFGS-B        5     1.1902e+00 ± 2.64e-01 18.5 ± 5.0           100.0                1.4996e+01           [-10,10]  
trust-constr    5     9.7972e-01 ± 1.50e-01 475.4 ± 205.3        100.0                1.1174e+01           [-10,10]  
Nelder-Mead     10    2.4809e+00 ± 2.11e-01 143.4 ± 113.4        90.0                 1.7880e+01           [-10,10]  
BFGS            10    2.6354e+00 ± 5.49e-01 31.8 ± 19.1          80.0                 1.8276e+01           [-10,10]  
L-BFGS-B        10    2.5758e+00 ± 2.41e-01 89.4 ± 52.3          100.0                1.9620e+01           [-10,10]  
trust-constr    10    2.5473e+00 ± 7.83e-01 1033.5 ± 924.0       100.0                1.6192e+01           [-10,10]  

## Exercício 4 - Função 61 - Hansen

A partir de diferentes pontos iniciais, encontrar ao menos dois dos mínimos globais.

$$
f_{61}(x_1, x_2) = \left[\sum_{i=1}^{4} (i+1)\cos(ix_1 + i + 1)\right] \times \left[\sum_{j=0}^{4} (j+1)\cos((j+2)x_2 + j + 1)\right]
$$

In [ ]:
import numpy as np
import time
from scipy.optimize import minimize
import matplotlib.pyplot as plt

def hansen_function(x):
    """Função Hansen com cálculo do valor"""
    x1, x2 = x[0], x[1]
    sum1 = sum((i+1) * np.cos(i*x1 + i + 1) for i in range(1, 5))
    sum2 = sum((j+1) * np.cos((j+2)*x2 + j + 1) for j in range(0, 5))
    return sum1 * sum2

# Configurações
n_runs = 10
bounds = [(-10, 10), (-10, 10)]  # Domínio reduzido para melhor convergência
known_minima = [
    np.array([-7.5899, -7.7083]),
    np.array([-7.5899, -1.5811]), 
    np.array([4.7020, -1.5811])
]

initial_points = [
    np.array([0.0, 0.0]),        # Centro
    np.array([-5.0, -5.0]),      # Perto do mínimo 1
    np.array([-5.0, 0.0]),       # Perto do mínimo 2
    np.array([5.0, 0.0]),        # Perto do mínimo 3
    np.random.uniform(-10, 10, 2), # Aleatório 1
    np.random.uniform(-10, 10, 2)  # Aleatório 2
]

methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS"},
    "L-BFGS-B": {"method": "L-BFGS-B", "bounds": bounds},
    "trust-constr": {"method": "trust-constr", "bounds": bounds}
}

results = []
found_minima = set()

for i, x0 in enumerate(initial_points):
    print(f"\n=== Teste com ponto inicial {i+1}: {x0} ===")
    
    for name, opts in methods.items():
        times = []
        solutions = []
        func_values = []
        successes = []
        minima_found = []
        
        for _ in range(n_runs):
            try:
                # Adiciona perturbação ao ponto inicial
                perturbed_x0 = x0 + np.random.normal(0, 0.5, size=2)
                
                start_time = time.time()
                res = minimize(hansen_function, perturbed_x0, **opts,
                             options={'maxiter': 5000, 'disp': False})
                elapsed_time = (time.time() - start_time) * 1000
                
                # Verifica se encontrou um dos mínimos conhecidos
                found = False
                for min_x in known_minima:
                    if np.linalg.norm(res.x - min_x) < 0.5:  # Tolerância de 0.5
                        minima_found.append(min_x.tolist())
                        found = True
                        break
                
                times.append(elapsed_time)
                solutions.append(res.x)
                func_values.append(res.fun)
                successes.append(res.success)
                
            except Exception as e:
                print(f"Erro no método {name}: {str(e)}")
                times.append(np.nan)
                solutions.append(np.array([np.nan, np.nan]))
                func_values.append(np.nan)
                successes.append(False)
        
        # Processa resultados
        valid_runs = [i for i in range(n_runs) if successes[i]]
        n_valid = len(valid_runs)
        
        if n_valid > 0:
            # Conta mínimos únicos encontrados
            unique_minima = []
            for min_x in minima_found:
                if min_x not in unique_minima:
                    unique_minima.append(min_x)
            
            avg_time = np.mean([times[i] for i in valid_runs])
            avg_solution = np.mean([solutions[i] for i in valid_runs], axis=0)
            avg_func = np.mean([func_values[i] for i in valid_runs])
            best_func = np.min([func_values[i] for i in valid_runs])
            best_solution = solutions[np.argmin([func_values[i] for i in valid_runs])]
            
            results.append({
                'Initial Point': f"{x0[0]:.1f}, {x0[1]:.1f}",
                'Method': name,
                'Avg Time (ms)': avg_time,
                'Success Rate': f"{n_valid/n_runs*100:.1f}%",
                'Avg f(x*)': avg_func,
                'Best f(x*)': best_func,
                'Best Solution': f"[{best_solution[0]:.6f}, {best_solution[1]:.6f}]",
                'Minima Found': len(unique_minima),
                'Unique Minima': unique_minima
            })
            
            # Atualiza found_minima
            for min_x in unique_minima:
                found_minima.add(tuple(min_x))

# Exibe resultados detalhados
print("\n=== RESULTADOS DETALHADOS ===")
print("="*150)
print("{:<15} {:<15} {:<15} {:<15} {:<15} {:<15} {:<25} {:<15} {:<20}".format(
    'Ponto Inicial', 'Método', 'Tempo Médio', 'Taxa Sucesso', 'Média f(x*)', 
    'Melhor f(x*)', 'Melhor Solução', 'Mínimos Encontrados', 'Mínimos Únicos'))
print("-"*150)

for res in results:
    print("{:<15} {:<15} {:<15.2f} {:<15} {:<15.6f} {:<15.6f} {:<25} {:<15} {:<20}".format(
        res['Initial Point'],
        res['Method'],
        res['Avg Time (ms)'],
        res['Success Rate'],
        res['Avg f(x*)'],
        res['Best f(x*)'],
        res['Best Solution'],
        res['Minima Found'],
        str(res['Unique Minima'])))

# Resumo de mínimos encontrados
print("\n=== MÍNIMOS GLOBAIS ENCONTRADOS ===")
print("="*60)
for i, min_x in enumerate(known_minima):
    found = any(np.allclose(min_x, found_min, atol=0.5) for found_min in found_minima)
    print(f"Mínimo {i+1}: [{min_x[0]:.4f}, {min_x[1]:.4f}] - {'Encontrado' if found else 'Não encontrado'}")



In [ ]:
=== RESULTADOS DETALHADOS ===
======================================================================================================================================================
Ponto Inicial   Método          Tempo Médio     Taxa Sucesso    Média f(x*)     Melhor f(x*)    Melhor Solução            Mínimos Encontrados Mínimos Únicos      
------------------------------------------------------------------------------------------------------------------------------------------------------
0.0, 0.0        Nelder-Mead     7.79            100.0%          -52.987862      -153.316446     [-0.350879, -0.800326]    0               []                  
0.0, 0.0        BFGS            6.10            100.0%          -41.298096      -153.316448     [-0.350845, -0.800321]    0               []                  
0.0, 0.0        L-BFGS-B        8.40            100.0%          -23.448111      -45.592568      [-3.657797, -7.083506]    0               []                  
0.0, 0.0        trust-constr    204.29          100.0%          -61.552906      -169.587624     [-1.306708, 4.858057]     0               []                  
-5.0, -5.0      Nelder-Mead     5.80            100.0%          -9.584266       -11.446633      [-5.800417, -5.461405]    0               []                  
-5.0, -5.0      BFGS            6.69            100.0%          -18.219428      -47.271789      [-7.589893, -8.794063]    0               []                  
-5.0, -5.0      L-BFGS-B        6.66            100.0%          -22.368651      -54.280724      [-5.090999, -7.083506]    0               []                  
-5.0, -5.0      trust-constr    152.92          100.0%          -31.312482      -169.587624     [-1.306708, -1.425128]    0               []                  
-5.0, 0.0       Nelder-Mead     5.99            100.0%          -23.515280      -54.280723      [-5.091027, -0.800296]    0               []                  
-5.0, 0.0       BFGS            5.41            100.0%          -31.713980      -54.280724      [-5.090999, -0.800321]    0               []                  
-5.0, 0.0       L-BFGS-B        6.25            100.0%          -19.170209      -49.410095      [-1.306708, -5.461401]    0               []                  
-5.0, 0.0       trust-constr    148.99          100.0%          -27.884798      -54.280724      [-5.090999, -0.800321]    0               []                  
5.0, 0.0        Nelder-Mead     9.50            100.0%          -92.821971      -153.316447     [5.932322, -0.800304]     0               []                  
5.0, 0.0        BFGS            6.13            100.0%          -73.984503      -169.587624     [4.976478, -1.425128]     1               [[4.702, -1.5811]]  
5.0, 0.0        L-BFGS-B        7.59            100.0%          -86.819225      -153.316448     [5.932341, -0.800321]     0               []                  
5.0, 0.0        trust-constr    136.16          100.0%          -95.646396      -153.316448     [5.932341, -0.800321]     0               []                  
-4.7, -3.9      Nelder-Mead     5.33            100.0%          -9.892109       -13.520353      [-5.090964, -3.003155]    0               []                  
-4.7, -3.9      BFGS            5.91            100.0%          -22.962738      -71.273945      [-2.202878, -0.800321]    0               []                  
-4.7, -3.9      L-BFGS-B        6.98            100.0%          -51.060307      -169.587624     [-7.589893, -1.425128]    1               [[-7.5899, -1.5811]]
-4.7, -3.9      trust-constr    170.29          100.0%          -9.242790       -11.830812      [-2.933579, -5.461401]    0               []                  
4.8, 6.2        Nelder-Mead     5.68            100.0%          -46.529097      -112.231505     [4.976480, 6.087783]      0               []                  
4.8, 6.2        BFGS            4.69            100.0%          -102.459356     -153.316448     [5.932341, 5.482864]      0               []                  
4.8, 6.2        L-BFGS-B        5.82            100.0%          -82.003054      -169.587624     [4.976478, 4.858057]      0               []                  
4.8, 6.2        trust-constr    183.80          100.0%          -82.980165      -169.587624     [4.976478, 4.858057]      0               []                  

=== MÍNIMOS GLOBAIS ENCONTRADOS ===
============================================================
Mínimo 1: [-7.5899, -7.7083] - Não encontrado
Mínimo 2: [-7.5899, -1.5811] - Encontrado
Mínimo 3: [4.7020, -1.5811] - Encontrado